In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import cross_validation

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.utils import shuffle


In [3]:
data = pd.read_csv('cancer.dat', header = None, names = ['ID', 'Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin', 'Normal Nucleoli','Mitoses','Class'],skiprows=[23,40,139,145,158,164,235,249,275,292,294,297,315,321,411,617]) 
X = data.values[:,1:10]
y = pd.get_dummies(data['Class']).values
X_train, X_test, y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.2)

In [4]:
x = tf.placeholder("float",[None,9])
y = tf.placeholder("float",[None,2])

In [5]:
#Set hyperparameters
learning_rate = .01 #how fast we update the weights, .01 is a known decent size
training_iteration = 30
batch_size = 10
display_step = 2

In [ ]:
tf.random_normal?

In [6]:
W = tf.Variable(tf.zeros([9,2]))
b = tf.Variable(tf.zeros([2]))

In [7]:
#Regression model
with tf.name_scope("Wx_b") as scope:
    model = tf.nn.relu(tf.matmul(x,W) +b) #relu activation

#Add summary ops to collect data

w_h = tf.summary.histogram("weights",W)
b_h = tf.summary.histogram("biases",b)

In [ ]:
#Next we make a scope to visualize the cost or loss function

with tf.name_scope("cost_function") as scope:
    #Minimize using cross entropy
    cost_function = -tf.reduce_sum(y*tf.log(model))
    tf.summary.scalar("cost_function",cost_function)

In [ ]:
with tf.name_scope("train") as scope:
    #Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=.01).minimize(cost_function)
    #tune hyperparameters later...
    
    

In [ ]:
#Initialize the variables
init = tf.initialize_all_variables()
#Merge all summaries into one operator
merged_summary_op = tf.summary.merge_all()

This is the part I keep having trouble with. Training time

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Change this to a location on your computer
    summary_writer = tf.summary.FileWriter('/Users/noahkasmanoff/MachLearn/hw3/', graph_def=sess.graph_def)

    # Training cycle
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(len(X_train)/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys =  next_batch(batch_size,data=X_train,labels=y_train)
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})/total_batch


In [ ]:
            
            
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute the average loss
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            # Write logs for each iteration
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        # Display logs per iteration step
        if iteration % display_step == 0:
            print("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Tuning completed!")

    # Test the model, still in session! 
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: y_test}))

In [ ]:
tf.placeholder?

In [ ]:
import numpy as np

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)



In [ ]:
next(10,X_train)